<a href="https://colab.research.google.com/github/SonDongBin/python-file/blob/master/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_20220629.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

In [7]:
text.split('\n')

['경마장에 있는 말이 뛰고 있다', '그의 말이 법이다', '가는 말이 고와야 오는 말이 곱다', '']

In [8]:
tokenizer.texts_to_sequences(['경마장에 있는 말이 뛰고 있다'])

[[2, 3, 1, 4, 5]]

In [11]:
len(sequences)

11

In [12]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [13]:
max(len(i) for i in sequences)
# [len(i) for i in sequences]

# a = []
# for i in sequences:
#   a.append(len(i))

6

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
text_g = pd.read_csv('ChatbotData.csv')
text_g = text_g.A.copy()
text_g.head()

In [ ]:
text_g.shape
pd.read_csv('ChatbotData.csv')['label'].value_counts().plot(kind='bar')

In [ ]:
list(text_g)[:10]
tokenizer_g = Tokenizer()
tokenizer_g.fit_on_texts(list(text_g))
vocab_size_g = len(tokenizer_g.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size_g)

In [ ]:
sequences_g = list()
for line in text_g: # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer_g.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences_g.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences_g))

print(sequences_g)
max_len_g = max(len(l) for l in sequences_g) 
print('샘플의 최대 길이 : {}'.format(max_len_g))

In [ ]:
sequences_g = pad_sequences(sequences_g, maxlen=max_len_g, padding='pre')
print(sequences_g)

In [ ]:
sequences_g = np.array(sequences_g)
X_chatbot_g = sequences_g[:,:-1]
y_chatbot_g = sequences_g[:, -1]

In [ ]:
print('X:', X_chatbot_g)
print('y:', y_chatbot_g)

In [ ]:
print('전체 y의 개수:', len(y_chatbot_g))
print('고유값 y의 개수:', np.unique(y_chatbot_g)[-1]+1)

In [ ]:
y_one_g = to_categorical(y_chatbot_g, num_classes=vocab_size_g)
print("one_hot_vector y:", y_one_g)

In [ ]:
# model design
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [ ]:
embedding_dim = 32
hidden_units = 64

model_g = Sequential()
model_g.add(Embedding(vocab_size_g, embedding_dim))
model_g.add(SimpleRNN(hidden_units))
model_g.add(Dense(vocab_size_g, activation='softmax'))
model_g.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_g.fit(X_chatbot_g, y_one_g, epochs=200, verbose=2)

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = currentword
    sentence = ''

    # n번 반복
    for  in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model_g, tokenizer_g, '너는', 5))